# Experimenting to fix cracks & folds in basil  
Using the Optimizer to fix cracks & folds in basil

In [7]:
!mkdir -p ~/.cloudvolume/secrets
!cp /notebooks/*.json ~/.cloudvolume/secrets/
!pip install cloud-volume
!pip install scikit-image

  Using cached https://files.pythonhosted.org/packages/96/39/dac6a9da64502176b00e445a591d1b6a23484fb585330073a4f965d16f97/cloud_volume-0.23.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/93/24/6ab1df969db228aed36a648a8959d1027099ce45fad67532b9673d533318/tqdm-4.23.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5b/c3/9b947e301e19bea75dc8c1fd3710eed5d2b31aa13ae13d5e38e891f784cc/protobuf-3.5.2.post1-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ad/0d/14f29bebb6f93362da511dd4299d0dee2fb19d294657ae2c1f4adfb930a6/boto3-1.7.28-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4b/cf/050247dcd8000a8f2413d7932281991acba5bb72270f5f39d0a22c9b3074/json5-0.5.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
  Using cached https://files.pythonh

    100% |################################| 51kB 9.0MB/s eta 0:00:01
    100% |################################| 51kB 11.7MB/s ta 0:00:01
    100% |################################| 92kB 11.0MB/s ta 0:00:01
    100% |################################| 71kB 12.2MB/s ta 0:00:01
    100% |################################| 153kB 7.2MB/s eta 0:00:01
    100% |################################| 51kB 11.4MB/s ta 0:00:01
    100% |################################| 61kB 11.5MB/s ta 0:00:01
    100% |################################| 61kB 13.7MB/s ta 0:00:01
    100% |################################| 143kB 8.5MB/s eta 0:00:01
    100% |################################| 92kB 9.3MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/e6/35/f187bdf23be87092bd0f1200d43d23076cee4d0dec109f195173fd3ebc79/mock-2.0.0-py2.py3-none-any.whl
    100% |################################| 614kB 2.3MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/93/46/a389a65237d0520bb4a98fc

    100% |################################| 552kB 2.7MB/s eta 0:00:01
    100% |################################| 51kB 11.8MB/s ta 0:00:01
    100% |################################| 102kB 7.8MB/s ta 0:00:01
    100% |################################| 204kB 6.1MB/s eta 0:00:01
    100% |################################| 81kB 12.9MB/s ta 0:00:01
    100% |################################| 81kB 12.0MB/s ta 0:00:01
    100% |################################| 163kB 7.5MB/s eta 0:00:01
    100% |################################| 71kB 11.5MB/s ta 0:00:01
    100% |################################| 829kB 1.8MB/s eta 0:00:01
    100% |################################| 9.0MB 174kB/s eta 0:00:01    85% |###########################     | 7.7MB 61.6MB/s eta 0:00:01
    100% |################################| 225kB 6.6MB/s eta 0:00:01
  Running setup.py bdist_wheel for psutil ... done
  Stored in directory: /root/.cache/pip/wheels/fe/33/84/baea3d665de2d1af7e8f827f3883811bba5e4149443ccf8191
  Runnin

  Running setup.py bdist_wheel for blosc ... done
  Stored in directory: /root/.cache/pip/wheels/fa/2a/00/8ff3a1d3d423a3163b93917a72146e85c3a3484ba6574854a5
  Running setup.py bdist_wheel for nose2 ... done
  Stored in directory: /root/.cache/pip/wheels/66/65/52/e7e6215ebee79eff0f3a60db0e6e6f6d751c3ebbb7a52cdfbd
  Running setup.py bdist_wheel for Markdown ... done
  Stored in directory: /root/.cache/pip/wheels/34/2d/c2/75f4fd57c1b79e59f66388262ed37b69ceac77162988f8af63
  Running setup.py bdist_wheel for pandocfilters ... done
  Stored in directory: /root/.cache/pip/wheels/fc/a0/4c/ec26eaa3f2253144e29941bd4031c317a69595dd0b665a0779
  Running setup.py bdist_wheel for inflection ... done
  Stored in directory: /root/.cache/pip/wheels/9f/fa/4f/ef45a51543bd855e40494298824ad5e750109ad18fa80278ad
  Running setup.py bdist_wheel for gapic-google-cloud-error-reporting-v1beta1 ... done
  Stored in directory: /root/.cache/pip/wheels/19/d9/9e/5d0f178712d65c20ef2e2573b4390f94864819fe0b29d91c8f
  Run

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
from cloudvolume import CloudVolume
from cloudvolume.lib import Bbox, Vec
import numpy as np
import torch
from torch.autograd import Variable
from optimize import Optimizer
from copy import deepcopy
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def use_src_mask_and_one_dst():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_offset = np.array([102716, 107077, 526])
    full_size = np.array([2048, 2048, 1])
    full_bbox = Bbox(full_offset, full_offset+full_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_bbox))
    dst_bbox = src_bbox + [0,0,1]
    mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_bbox))
    print(src_bbox)
    print(dst_bbox)
    print(mask_bbox)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_with_mask", mip=img_mip, 
                        info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    dst[dst_bbox.to_slices()] = src[dst_bbox.to_slices()]
    src_img = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    dst_img = src[dst_bbox.to_slices()][:,:,0,0] / 255.0

    fold = folds[mask_bbox.to_slices()][:,:,0,0] / 255.0
    crack = cracks[mask_bbox.to_slices()][:,:,0,0] / 255.0
    mask = resize(np.minimum(fold, crack), src_bbox.size3()[:2])
    field = flow.process(src_img, [dst_img], mask, [np.ones_like(dst_img)])
    pred = flow.render(src_img, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

Bbox([25679, 26769, 526],[26191, 27281, 527])
Bbox([25679, 26769, 527],[26191, 27281, 528])
Bbox([3209, 3346, 526],[3273, 3410, 527])


In [11]:
def use_masks_and_two_dsts():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [15]:
def use_masks_and_four_dsts():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-2,-1,1,2]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_four_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [32]:
def use_filtered_mask_and_two_dsts():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_filtered_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask *= (src_mask < 1/255.0) | (src_mask > 10/255.0)
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [34]:
def use_filtered_mask_and_two_dsts2():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_filtered_mask_two_dsts2", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask *= (src_mask < 4/255.0) | (src_mask > 10/255.0)
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [11]:
def use_masks_and_two_dsts_mip1():
    img_mip = 1
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_mip1", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field
    

In [13]:
def use_masks_and_two_dsts_mip0():
    img_mip = 0
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_mip0", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field
    

In [24]:
def use_masks_and_two_dsts_crack():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102320, 111106, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask = np.ones_like(src_mask) * ((src_mask < 6/255.0) | (src_mask > 20/255.0))
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field

In [24]:
def use_gt_masks_and_two_dsts():
    mip = 2
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=mip, 
                                                                    parallel=2, fill_missing=True)
    mask = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth", mip=mip,
                                                                     parallel=2, fill_missing=True)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v02_ground_truth_masks_two_dsts", 
          mip=mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(102320, 111106, 526), (102716, 107077, 526)]
    full_src_size = np.array([2048, 2048, 1])
    for full_src_offset in offsets:
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox in dst_bboxes:
            dst_slices = dst_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_mask = (mask[dst_slices][:,:,0,0] == 0) / 1.0
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_mask = (mask[src_bbox.to_slices()][:,:,0,0] == 0) / 1.0

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)

In [26]:
def without_masks_and_two_dsts():
    mip = 2
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=mip, 
                                                                    parallel=2, fill_missing=True)
    mask = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth", mip=mip,
                                                                     parallel=2, fill_missing=True)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v02_without_masks_two_dsts", 
          mip=mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(102320, 111106, 526), (102716, 107077, 526)]
    full_src_size = np.array([2048, 2048, 1])
    for full_src_offset in offsets:
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox in dst_bboxes:
            dst_slices = dst_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_image = src[dst_slices][:,:,0,0] / 255.0
            dst_images.append(dst_image)
            # dst_mask = (mask[dst_slices][:,:,0,0] == 0) / 1.0
            dst_mask = np.ones_like(dst_image)
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        # src_mask = (mask[src_bbox.to_slices()][:,:,0,0] == 0) / 1.0
        src_mask = np.ones_like(src_image)

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)

In [44]:
def use_masks_and_two_dsts_more_locations():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(191721, 139068, 528),    # consecutive folds
               Vec(196097, 177069, 526),    # consecutive folds
               Vec(179301, 81627, 529),     # big crack
               Vec(167572, 73149, 522),     # track mark, no masks
               Vec(146350, 204341, 524),    # consecutive cracks
               Vec(130159, 162051, 525)     # consecutive cracks
              ]
    full_src_size = np.array([2048, 2048, 1])
    for full_src_offset in offsets:
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
            crack_max = np.max(dst_crack * (dst_crack < 20/255.0))
            crack_min = max(1, crack_max-3)
            dst_crack = np.ones_like(dst_crack) * ((dst_crack < crack_min) | (dst_crack > crack_max))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        crack_max = np.max(src_crack * (src_crack < 20/255.0))
        crack_min = max(1, crack_max-3)
        src_crack = np.ones_like(src_crack) * ((src_crack < crack_min) | (src_crack > crack_max))
        src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    